In [1]:
import numpy as np
import pandas as pd
import sys
import os

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
os.getcwd()

'/Users/harshbaberwal/Desktop/Projects/git_repos/tattoo_style_tagger/development'

In [4]:
sys.path.append("/Users/harshbaberwal/Desktop/Projects/git_repos/tattoo_style_tagger")

In [5]:
style_info = pd.read_csv("../data/style_queries_w_description.csv")
style_info.head()

,style,style_query,style_desc
0,abstract,abstract,"Just like the art movement it is inspired by, ..."
1,anime,anime,NaN
2,black_and_gray,black_and_gray,Shading is key when it comes to black and grey...
3,blackwork,blackwork,Black and bold is what makes Blackwork tattoos...
4,chicano,chicano,Often using black and grey and an illustrative...


In [6]:
styles_for_test = style_info.head(3)
styles_for_test

,style,style_query,style_desc
0,abstract,abstract,"Just like the art movement it is inspired by, ..."
1,anime,anime,NaN
2,black_and_gray,black_and_gray,Shading is key when it comes to black and grey...


## Meta Data Pull

In [8]:
from lib.data_pull.meta_data_pull import read_tattoos_meta_data

In [9]:
all_styles_meta_data = read_tattoos_meta_data(styles_for_test)

Reading style abstract
Done reading meta data for style abstract 
Reading style anime
Done reading meta data for style anime 
Reading style black_and_gray
Done reading meta data for style black_and_gray 


In [17]:
all_styles_meta_data.head(3)

,tattoo_id,image_url,styles,searched_style_query
0,1011413,https://d1kq2dqeox7x40.cloudfront.net/images/p...,"['abstract', 'black_and_gray', 'blackwork', 'f...",abstract
1,1053382,https://d1kq2dqeox7x40.cloudfront.net/images/p...,"['abstract', 'black_and_gray', 'blackwork', 'h...",abstract
2,531019,https://d1kq2dqeox7x40.cloudfront.net/images/p...,"['abstract', 'ornamental', 'watercolor']",abstract


## Meta Data Pull Testing Code

In [26]:
import requests

In [32]:
page_limit: int = 10
page = 1
style_query = "abstract"
base_url: str = "https://backend-api.tattoodo.com/api/v2/search/posts?"

api_url: str = base_url+f"style={style_query}&page={1}&limit={10}"

response = requests.get(api_url)
response_dict: dict = json.loads(response.text)

In [34]:
response_dict

{'data': [{'id': 1011413,
   'description': '#sssblcakwork #sandrosecchin #tattoo #tattoodo #tattoodobr #inked #inkfreakz #besttattoo #uktta #tttism #darkartists #blacktattooart #tattrx #blackwork #tattooistartmagazine #besttattoo #tguest #inspirationtattoo #tattoosofinstagram #toptattooartist #blackworker #blackworkersubmission #blxckink #radtattoos #inkjunkeyz #thebesttattooartists #londontattoo #londonart #blkttt #ttt_ist #btattooing #sketch #sketchtattoo ',
   'image': {'url': 'https://d1kq2dqeox7x40.cloudfront.net/images/posts/20190527_a6pryHC2cRCXxbZ.png',
    'width': 1010,
    'height': 1515},
   'counts': {'likes': 3, 'comments': 0, 'pins': 8},
   'created_at': '2019-05-27T11:52:27+00:00',
   'updated_at': '2023-08-24T09:25:52+00:00',
   'uploader': {'id': 15227,
    'name': 'Sandro Secchin',
    'username': 'sandrosecchin',
    'pronouns': None,
    'image_url': 'https://d1kq2dqeox7x40.cloudfront.net/images/profile_images/20220714_Pwo0anMibA2a67E.webp',
    'user_type': 'arti

In [44]:
get_tattoos_meta_data_from_page(response_dict['data'])

,tattoo_id,image_url,styles
0,1011413,https://d1kq2dqeox7x40.cloudfront.net/images/p...,"[abstract, black_and_gray, blackwork, fine_lin..."
1,1053382,https://d1kq2dqeox7x40.cloudfront.net/images/p...,"[abstract, black_and_gray, blackwork, horror, ..."
2,531019,https://d1kq2dqeox7x40.cloudfront.net/images/p...,"[abstract, ornamental, watercolor]"
3,80476,https://d1kq2dqeox7x40.cloudfront.net/images/p...,"[abstract, watercolor]"
4,830251,https://d1kq2dqeox7x40.cloudfront.net/images/p...,"[abstract, illustrative, trashpolka]"
5,997456,https://d1kq2dqeox7x40.cloudfront.net/images/p...,"[abstract, blackwork, fine_line, illustrative]"
6,531048,https://d1kq2dqeox7x40.cloudfront.net/images/p...,"[abstract, black_and_gray]"
7,531029,https://d1kq2dqeox7x40.cloudfront.net/images/p...,"[abstract, new_school, watercolor]"
8,1700398,https://d1kq2dqeox7x40.cloudfront.net/images/p...,"[abstract, blackwork, fine_line, geometric, il..."
9,1614667,https://d1kq2dqeox7x40.cloudfront.net/images/p...,"[abstract, new_school, watercolor]"


## NEw

In [21]:
all_styles_meta_data.to_csv("temp/all_styles_meta_data.csv", index=False)

In [23]:
all_styles_meta_data = pd.read_csv("temp/all_styles_meta_data.csv")

In [25]:
type(all_styles_meta_data.styles[0])

str

## Meta Data Processing

## 1

In [29]:
from lib.data_pull.meta_data_processing import process_tattoo_meta_data_to_long_form

In [31]:
tattoo_meta_data_exploded = process_tattoo_meta_data_to_long_form(all_styles_meta_data)

In [33]:
tattoo_meta_data_exploded.head()

,tattoo_id,image_url,styles
0,1011413,https://d1kq2dqeox7x40.cloudfront.net/images/p...,abstract
0,1011413,https://d1kq2dqeox7x40.cloudfront.net/images/p...,black_and_gray
0,1011413,https://d1kq2dqeox7x40.cloudfront.net/images/p...,blackwork
0,1011413,https://d1kq2dqeox7x40.cloudfront.net/images/p...,fine_line
0,1011413,https://d1kq2dqeox7x40.cloudfront.net/images/p...,horror


#### Sanity Check

In [36]:
all_styles_meta_data.tattoo_id.nunique() - tattoo_meta_data_exploded.tattoo_id.nunique()

588

In [38]:
all_styles_meta_data[all_styles_meta_data['styles'] == "[]"].tattoo_id.nunique()

588

## 2

In [41]:
from lib.data_pull.meta_data_processing import filter_out_tattoos

In [43]:
tattoo_meta_data_exploded_filtered = filter_out_tattoos(tattoo_meta_data_exploded)

In [45]:
tattoo_meta_data_exploded_filtered.head()

,tattoo_id,image_url,styles
0,1011413,https://d1kq2dqeox7x40.cloudfront.net/images/p...,abstract
1,1011413,https://d1kq2dqeox7x40.cloudfront.net/images/p...,black_and_gray
2,1011413,https://d1kq2dqeox7x40.cloudfront.net/images/p...,blackwork
3,1011413,https://d1kq2dqeox7x40.cloudfront.net/images/p...,fine_line
4,1011413,https://d1kq2dqeox7x40.cloudfront.net/images/p...,horror


#### Sanity Check

In [48]:
tattoo_meta_data_exploded.tattoo_id.nunique() - tattoo_meta_data_exploded_filtered.tattoo_id.nunique()

122

In [54]:
tattoo_meta_data_exploded.groupby(
    'image_url', as_index=False
    )['styles'].nunique().query("styles > 7").shape

(122, 2)

## 3

In [56]:
from lib.data_pull.meta_data_processing import handle_duplicates

In [58]:
tattoo_meta_data_exploded_filtered_final = handle_duplicates(tattoo_meta_data_exploded_filtered)

image_url    0
styles       0
tattoo_id    0
dtype: int64


#### Sanity Check

In [68]:
tattoo_meta_data_exploded_filtered.image_url.nunique() - tattoo_meta_data_exploded_filtered_final.image_url.nunique()

0

In [70]:
tattoo_meta_data_exploded_filtered.tattoo_id.nunique() - tattoo_meta_data_exploded_filtered_final.tattoo_id.nunique()

0

In [79]:
tattoo_meta_data_exploded_filtered.groupby(
    'image_url', as_index=False
    )['tattoo_id'].nunique().query("tattoo_id > 1").shape

(0, 2)